In [ ]:
import os
import json
import xml.etree.ElementTree as ET
from datetime import datetime

import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import pystac
import sys
from shapely.geometry import mapping, box
from constants import DEFAULT_START_DATE, DEFAULT_END_DATE
import environ
env = environ.Env()
env.read_env()
base_dir=env("BASE_DIR")
sys.path.append(base_dir)
http_base=env("http_base")



In [56]:


corestack_dir = os.path.join(base_dir, "Corestack Catalogs")
gobindpur_dir = os.path.join(corestack_dir, "gobindpur")
raster_dir = os.path.join(gobindpur_dir, "raster")
vector_dir = os.path.join(gobindpur_dir, "vector")

os.makedirs(raster_dir, exist_ok=True)
os.makedirs(vector_dir, exist_ok=True)


raster_filename = env("raster_filename")
vector_filename = env("vector_filename")
raster_path = os.path.join(base_dir, raster_filename)
vector_path = os.path.join(base_dir, vector_filename)

raster_thumb = os.path.join(raster_dir, "thumbnail.png")
vector_thumb = os.path.join(vector_dir, "thumbnail.png")

raster_style_file = os.path.join(base_dir, "style_file.qml")
vector_style_file = os.path.join(base_dir, "swb_style.qml")


In [41]:
def extract_raster_dates_from_filename(filename):
    try:
        parts = filename.split('_')
        start_date = datetime.strptime(parts[2], "%Y-%m-%d")
        end_date = datetime.strptime(parts[3], "%Y-%m-%d")
    except Exception as e:
        print(f" Failed to extract raster dates from filename '{filename}': {e}")
        start_date = DEFAULT_START_DATE
        end_date = DEFAULT_END_DATE
    return start_date, end_date


In [42]:
def parse_qml_classes(qml_path):
    tree = ET.parse(qml_path)
    root = tree.getroot()
    classes = []

    for entry in root.findall(".//paletteEntry"):
        class_info = {}
        for attr_key, attr_value in entry.attrib.items():
            if attr_key == "value":
                try:
                    class_info[attr_key] = int(attr_value)
                except ValueError:
                    class_info[attr_key] = attr_value
            else:
                class_info[attr_key] = attr_value
        classes.append(class_info)
    return classes

In [44]:
def generate_vector_thumbnail_single_style(vector_path, out_path, fill_color="#CCCCCC", outline_color="#333333"):

    gdf = gpd.read_file(vector_path)

    if gdf.empty:
        raise ValueError("GeoDataFrame is empty")

    if gdf.crs is None or gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)

    fig, ax = plt.subplots(figsize=(3, 3))
    fig.patch.set_facecolor("white")
    ax.set_facecolor("white")

    gdf.plot(ax=ax, color=fill_color, edgecolor=outline_color, linewidth=0.5)

    ax.set_xlim(*gdf.total_bounds[[0, 2]])
    ax.set_ylim(*gdf.total_bounds[[1, 3]])
    ax.axis("off")
    plt.savefig(out_path, dpi=150, bbox_inches="tight", pad_inches=0, facecolor=fig.get_facecolor())
    plt.close()


In [45]:
def generate_raster_thumbnail(tif_path, out_path):
    with rasterio.open(tif_path) as src:
        arr = src.read(1)
    plt.figure(figsize=(3, 3))
    plt.imshow(arr, cmap="tab20")
    plt.axis('off')
    plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
    plt.close()

def generate_vector_thumbnail(vector_path, out_path):
    gdf = gpd.read_file(vector_path)
    if gdf.crs is None or gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)
    fig, ax = plt.subplots(figsize=(3, 3))
    fig.patch.set_facecolor("white")
    ax.set_facecolor("white")
    gdf.plot(ax=ax, color="lightblue", edgecolor="blue", linewidth=0.5)
    ax.axis('off')
    plt.savefig(out_path, dpi=150, bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor())
    plt.close()


In [46]:
def generate_legend_image(legend_data, output_path):
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches

    labels = [entry.get("label", str(entry["value"])) for entry in legend_data]
    colors = [entry["color"] for entry in legend_data]

    fig_height = max(1, len(labels) * 0.4)
    fig, ax = plt.subplots(figsize=(3, fig_height))
    ax.axis("off")

    handles = [mpatches.Patch(color=color, label=label) for color, label in zip(colors, labels)]
    ax.legend(handles=handles, loc="center left", frameon=False)

    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()


In [47]:
def create_raster_item():
    start_date, end_date = extract_raster_dates_from_filename(raster_filename)
    with rasterio.open(raster_path) as src:
        bounds = src.bounds
        geom = mapping(box(*bounds))
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]

    generate_raster_thumbnail(raster_path, raster_thumb)
    style_info = parse_qml_classes(raster_style_file)

   
    style_json_path = os.path.join(raster_dir, "legend.json")
    with open(style_json_path, "w") as f:
        json.dump(style_info, f, indent=2)

    
    legend_img_path = os.path.join(raster_dir, "legend.png")
    generate_legend_image(style_info, legend_img_path)

    
    item = pystac.Item(
        id=env("id_raster"),
        geometry=geom,
        bbox=bbox,
        datetime=start_date,
        properties={
            "proj:epsg": env("proj"),
            "title":env("title"),
            "description": env("description"),
            "lulc:classes": style_info,
            "start_datetime": start_date.isoformat(),
            "end_datetime": end_date.isoformat()
        }
    )

    item.add_asset("data", pystac.Asset(
        href=f"{http_base}/{raster_filename}",
        media_type=pystac.MediaType.GEOTIFF,
        roles=["data"],
        title="Raster Layer"
    ))
    item.add_asset("thumbnail", pystac.Asset(
        href=f"{http_base}/raster/thumbnail.png",
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail"],
        title="Raster Thumbnail"
    ))
    item.add_asset("legend_image", pystac.Asset(
        href=f"{http_base}/raster/legend.png",
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail", "metadata"],
        title="Legend Image"
    ))
    item.add_asset("legend", pystac.Asset(
        href=f"{http_base}/raster/legend.json",
        media_type=pystac.MediaType.JSON,
        roles=["metadata"],
        title="Legend JSON"
    ))


    item.set_self_href(os.path.join(raster_dir, "item.json"))
    item.save_object()
    return item


In [48]:
raster_item=create_raster_item()

In [49]:
def create_vector_item():
    start_date = DEFAULT_START_DATE
    end_date = DEFAULT_END_DATE

    gdf = gpd.read_file(vector_path)
    geom = mapping(gdf.union_all())
    bounds = gdf.total_bounds
    bbox = [float(b) for b in bounds]

    generate_vector_thumbnail(vector_path, vector_thumb)
    style_info = parse_qml_classes(vector_style_file)
    style_json_path = os.path.join(vector_dir, "style.json")
    with open(style_json_path, "w") as f:
        json.dump(style_info, f, indent=2)

    item = pystac.Item(
        id=env("id_vector"),
        geometry=geom,
        bbox=bbox,
        datetime=start_date,
        properties={
            "proj:epsg": env("proj"),
            "title": env("title_Vector"),
            "description": env("description_vector"),
            "style": style_info,
            "start_datetime": start_date.isoformat(),
            "end_datetime": end_date.isoformat()
        }
    )
    item.add_asset("data", pystac.Asset(
        href=f"{http_base}/{vector_filename}",
        media_type=pystac.MediaType.GEOJSON,
        roles=["data"],
        title="Vector Layer"
    ))
    item.add_asset("thumbnail", pystac.Asset(
        href=f"{http_base}/vector/thumbnail.png",
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail"],
        title="Vector Thumbnail"
    ))
    item.add_asset("style", pystac.Asset(
        href=f"{http_base}/vector/swb_style_thumbnail.json",
        media_type=pystac.MediaType.PNG,
        roles=["style"],
        title="Vector style thumbnail"
    ))
    item.set_self_href(os.path.join(vector_dir, "item.json"))
    item.save_object()
    return item


In [50]:

vector_item = create_vector_item()


In [70]:

gobindpur_catalog = pystac.Catalog(
    id=env("id_main"),
    title=env("title_main"),
    description=env("description_main")
)
gobindpur_catalog.add_item(create_raster_item())
gobindpur_catalog.add_item(create_vector_item())
gobindpur_catalog.set_self_href(os.path.join(gobindpur_dir, "catalog.json"))

corestack_catalog = pystac.Catalog(
    id="corestack",
    title="CorestackCatalogs",
    description="Root catalog containing all subcatalogs"
)
corestack_catalog.add_child(gobindpur_catalog)
corestack_catalog.set_self_href(os.path.join(corestack_dir, "catalog.json"))
corestack_catalog.normalize_and_save(corestack_dir, catalog_type=pystac.CatalogType.SELF_CONTAINED)

print(f" Root STAC Catalog created at: {os.path.join(corestack_dir, 'catalog.json')}")


 Root STAC Catalog created at: /home/vishnu/corestack_STAC/STAC_common/data/Corestack Catalogs/catalog.json
